级联分类器：Haar

In [ ]:
import cv2
import numpy as np

face_detector = cv2.CascadeClassifier('haar.xml')


计算单通道直方图相似度

In [32]:
def hist_similar(im1, im2):
    hist1 = cv2.calcHist([im1], [0], None, [256], [0.0, 255.0])
    hist2 = cv2.calcHist([im2], [0], None, [256], [0.0, 255.0])
    degree = 0
    for i in range(len(hist1)):
        if hist1[i] != hist2[i]:
            degree = degree + (1 - abs(hist1[i] - hist2[i]) / max(hist1[i], hist2[i]))
        else:
            degree = degree + 1
    degree = degree / len(hist1)
    return degree

SyntaxError: unexpected character after line continuation character (4062882633.py, line 7)

计算三通道直方图相似度

In [ ]:
def compare_hist(cap, face):
    shape = face.shape
    cap = cv2.resize(cap, shape)
    sub_cap = cv2.split(cap)
    sub_face = cv2.split(face)
    sub_data = 0
    for im1, im2 in zip(sub_cap, sub_face):
        sub_data += hist_similar(im1, im2)
    sub_data = sub_data / 3
    return sub_data

识别人脸，并裁剪图片
1. 裁剪后图片为正方形，大小在190-260之间
2. 如果未检测到人脸，输出是480*640的空数组

In [ ]:
def face_locate(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)
    shape = img.shape # (480, 640, 3)
    pad = 30
    cut_img = np.zeros(shape)
    w0 = 0
    for (x, y, w, h) in faces:
        if w >= w0:
            cv2.rectangle(img, (x, y), (x+w, y+w), (0, 255, 0))
            cut_img = img[(x-pad):(x+w+pad), (y-pad):(y+w+pad)]
            w0 = w
    return cut_img

识别身份

In [ ]:
def face_reg(img):
    names = ['Tianyou Bai', 'Zhiyi Wang', 'Qihang Yang', 'Aoqing Wang', 'Stranger']
    face0 = cv2.imread('pics/bai.jpeg')
    face1 = cv2.imread('pics/zhiyi.jpeg')
    face2 = cv2.imread('pics/yang.jpeg')
    face3 = cv2.imread('pics/aoqing.jpeg')
    # shape = img.shape # 识别不到人脸为(480, 640, 3)
    # print(shape)
    sim0 = compare_hist(img, face0)
    sim1 = compare_hist(img, face1)
    sim2 = compare_hist(img, face2)
    sim3 = compare_hist(img, face3)
    if sim0 >= 0.48: idx = 0
    elif sim1 >= 0.48: idx = 1
    elif sim2 >= 0.48: idx = 2
    elif sim3 >= 0.48: idx = 3
    else:idx = 4
    return names[idx]

主程序

In [ ]:
def run():
    vd = cv2.VideoCapture(0)
    vd.set(6, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
    
    if vd.isOpened():
        print('camera on')
        while True:
            ready, frame = vd.read()
            if frame is None:
                print('error')
                break
            if ready:
                frame = cv2.flip(frame, 1)
                obj = face_locate(frame)
                name = face_reg(obj)
                if name != 'Stranger':
                    cv2.putText(frame, name, (220, 55), cv2.FONT_HERSHEY_TRIPLEX, 1.0, (255, 0, 0), 2)
                else:
                    cv2.putText(frame, name, (240, 55), cv2.FONT_HERSHEY_TRIPLEX, 1.0, (0, 0, 255), 2)
                # print(name)
                cv2.imshow('source', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    vd.release()
                    cv2.destroyAllWindows()
                    break
        print('finished')

FACE IDENTIFIER

In [ ]:
if __name__ == '__main__':
    run()